# Agentic Underwriting Assistant

## Data Exploration and Analysis

First, we explore the datasets for the underwriting system. We'll examine policyholder demographics, claims history, policy details, and document corpus to understand the data structure and identify key patterns for risk assessment.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(" Libraries imported successfully!")
print(" Starting data exploration...")


 Libraries imported successfully!
 Starting data exploration...


In [2]:
# Load datasets
data_path = Path("../data/raw")

# Load tabular datasets
print(" Loading datasets...")
policyholders = pd.read_csv(data_path / "policyholder_demographics.csv")
claims = pd.read_csv(data_path / "historical_claims.csv")
policies = pd.read_csv(data_path / "policy_details.csv")

# Load document corpus
with open(data_path / "document_corpus.json", 'r', encoding='utf-8') as f:
    documents = json.load(f)

print(" All datasets loaded successfully!")
print(f" Policyholders: {len(policyholders):,}")
print(f" Claims: {len(claims):,}")
print(f" Policies: {len(policies):,}")
print(f" Documents: {len(documents)}")


 Loading datasets...
 All datasets loaded successfully!
 Policyholders: 1,000
 Claims: 2,000
 Policies: 1,494
 Documents: 6


## 1. Policyholder Demographics Analysis

Analyze the demographic characteristics of policyholders including age, gender, income, education, employment status, and credit scores. This section provides insights into the customer base and potential risk factors.


In [3]:
# Examine policyholder demographics
print(" Policyholder Demographics Overview")
print("=" * 50)
print(f"Total policyholders: {len(policyholders):,}")
print(f"Columns: {list(policyholders.columns)}")
print("\nFirst 5 rows:")
policyholders.head()


 Policyholder Demographics Overview
Total policyholders: 1,000
Columns: ['policyholder_id', 'age', 'gender', 'marital_status', 'annual_income', 'education_level', 'employment_status', 'credit_score', 'years_at_address', 'created_date']

First 5 rows:


,policyholder_id,age,gender,marital_status,annual_income,education_level,employment_status,credit_score,years_at_address,created_date
0,PH_000001,52,Male,Single,33.60,Master,Employed,573,14.4,2024-10-27 20:57:36.461722
1,PH_000002,42,Male,Single,64.65,PhD,Self-employed,537,2.0,2025-07-24 20:57:36.461722
2,PH_000003,54,Female,Single,142.60,Bachelor,Employed,680,4.1,2025-09-07 20:57:36.461722
3,PH_000004,67,Female,Married,36.84,High School,Retired,522,0.5,2025-05-02 20:57:36.461722
4,PH_000005,41,Male,Divorced,20.00,Master,Employed,608,1.1,2025-05-17 20:57:36.461722


In [4]:
# Summary statistics for numerical variables
print(" Summary Statistics - Policyholder Demographics")
print("=" * 60)
policyholders.describe()


 Summary Statistics - Policyholder Demographics


,age,annual_income,credit_score,years_at_address
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,44.904000,76.560590,644.012000,4.646400
std,14.178639,66.372364,98.524347,4.623952
min,18.000000,20.000000,360.000000,0.000000
25%,35.000000,30.990000,575.750000,1.300000
50%,45.000000,55.215000,645.000000,3.100000
75%,54.000000,94.707500,712.250000,6.300000
max,80.000000,500.000000,850.000000,27.600000


In [5]:
# Categorical variables analysis
print(" Categorical Variables Distribution")
print("=" * 50)

categorical_cols = ['gender', 'marital_status', 'education_level', 'employment_status']
for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print(policyholders[col].value_counts())
    print(f"Missing values: {policyholders[col].isnull().sum()}")


 Categorical Variables Distribution

GENDER:
gender
Male      509
Female    491
Name: count, dtype: int64
Missing values: 0

MARITAL_STATUS:
marital_status
Married     502
Single      306
Divorced    135
Widowed      57
Name: count, dtype: int64
Missing values: 0

EDUCATION_LEVEL:
education_level
High School    416
Bachelor       332
Master         195
PhD             57
Name: count, dtype: int64
Missing values: 0

EMPLOYMENT_STATUS:
employment_status
Employed         601
Self-employed    191
Retired          109
Unemployed        99
Name: count, dtype: int64
Missing values: 0


In [6]:
# Key variables analysis
print(" Key Variables Analysis")
print("=" * 40)

# Credit Score Analysis
print("\n CREDIT SCORE:")
print(f"Mean: {policyholders['credit_score'].mean():.1f}")
print(f"Median: {policyholders['credit_score'].median():.1f}")
print(f"Min: {policyholders['credit_score'].min()}")
print(f"Max: {policyholders['credit_score'].max()}")
print(f"Std: {policyholders['credit_score'].std():.1f}")

# Income Analysis (in thousands)
print(f"\n ANNUAL INCOME (in thousands):")
print(f"Mean: ${policyholders['annual_income'].mean():.2f}K")
print(f"Median: ${policyholders['annual_income'].median():.2f}K")
print(f"Min: ${policyholders['annual_income'].min():.2f}K")
print(f"Max: ${policyholders['annual_income'].max():.2f}K")

# Age Analysis
print(f"\n AGE:")
print(f"Mean: {policyholders['age'].mean():.1f} years")
print(f"Median: {policyholders['age'].median():.1f} years")
print(f"Min: {policyholders['age'].min()} years")
print(f"Max: {policyholders['age'].max()} years")


 Key Variables Analysis

 CREDIT SCORE:
Mean: 644.0
Median: 645.0
Min: 360
Max: 850
Std: 98.5

 ANNUAL INCOME (in thousands):
Mean: $76.56K
Median: $55.22K
Min: $20.00K
Max: $500.00K

 AGE:
Mean: 44.9 years
Median: 45.0 years
Min: 18 years
Max: 80 years


## 2. Claims Analysis

Examine historical claims data including claim amounts, types, severity levels, and status. This analysis helps identify patterns in claims behavior and assess risk factors associated with different types of claims.


In [7]:
# Examine claims data
print(" Claims Data Overview")
print("=" * 40)
print(f"Total claims: {len(claims):,}")
print(f"Columns: {list(claims.columns)}")
print("\nFirst 5 rows:")
claims.head()


 Claims Data Overview
Total claims: 2,000
Columns: ['claim_id', 'policyholder_id', 'claim_type', 'claim_amount', 'severity', 'status', 'claim_date', 'description']

First 5 rows:


,claim_id,policyholder_id,claim_type,claim_amount,severity,status,claim_date,description
0,CL_000001,PH_000558,Health,193.93,Low,Closed,2024-06-16 20:57:36.472984,Health claim - Low severity
1,CL_000002,PH_000664,Property,4380.91,High,Open,2024-11-18 20:57:36.472984,Property claim - High severity
2,CL_000003,PH_000931,Property,3286.37,High,Closed,2024-03-24 20:57:36.472984,Property claim - High severity
3,CL_000004,PH_000847,Auto,2714.83,High,Closed,2024-12-02 20:57:36.472984,Auto claim - High severity
4,CL_000005,PH_000074,Auto,1049.43,Low,Denied,2025-05-20 20:57:36.472984,Auto claim - Low severity


In [8]:
# Claims analysis
print(" Claims Analysis")
print("=" * 30)

# Claim amounts
print(f"\n CLAIM AMOUNTS:")
print(f"Mean: ${claims['claim_amount'].mean():.2f}")
print(f"Median: ${claims['claim_amount'].median():.2f}")
print(f"Min: ${claims['claim_amount'].min():.2f}")
print(f"Max: ${claims['claim_amount'].max():.2f}")
print(f"Total: ${claims['claim_amount'].sum():,.2f}")

# Claim types
print(f"\n CLAIM TYPES:")
print(claims['claim_type'].value_counts())

# Claim severity
print(f"\n CLAIM SEVERITY:")
print(claims['severity'].value_counts())

# Claim status
print(f"\n CLAIM STATUS:")
print(claims['status'].value_counts())


 Claims Analysis

 CLAIM AMOUNTS:
Mean: $1703.58
Median: $744.26
Min: $86.38
Max: $36264.67
Total: $3,407,153.05

 CLAIM TYPES:
claim_type
Auto          772
Property      511
Health        383
Life          222
Disability    112
Name: count, dtype: int64

 CLAIM SEVERITY:
severity
Low         988
Medium      593
High        307
Critical    112
Name: count, dtype: int64

 CLAIM STATUS:
status
Closed          1403
Open             199
Denied           199
Under Review     199
Name: count, dtype: int64


## 3. Policy Details Analysis

Review policy information including coverage amounts, premium calculations, deductibles, and policy types. This section analyzes the relationship between policy characteristics and risk factors.


In [9]:
# Examine policy details
print(" Policy Details Overview")
print("=" * 40)
print(f"Total policies: {len(policies):,}")
print(f"Columns: {list(policies.columns)}")
print("\nFirst 3 rows:")
policies.head(3)


 Policy Details Overview
Total policies: 1,494
Columns: ['policy_id', 'policyholder_id', 'policy_type', 'coverage_amount', 'premium_amount', 'status', 'start_date', 'end_date', 'deductible', 'coverage_limits']

First 3 rows:


,policy_id,policyholder_id,policy_type,coverage_amount,premium_amount,status,start_date,end_date,deductible,coverage_limits
0,POL_PH_000001_1,PH_000001,Auto,91.34,100,Active,2023-10-12 20:57:36.558125,2024-10-11 20:57:36.558125,2858.72,"{'per_incident': 73.07, 'annual': 91.34, 'life..."
1,POL_PH_000001_2,PH_000001,Home,58.06,100,Active,2024-12-14 20:57:36.558125,2025-12-14 20:57:36.558125,3348.45,"{'per_incident': 46.45, 'annual': 58.06, 'life..."
2,POL_PH_000002_1,PH_000002,Home,165.91,100,Active,2025-05-26 20:57:36.558125,2026-05-26 20:57:36.558125,4253.33,"{'per_incident': 132.73, 'annual': 165.91, 'li..."


In [10]:
# Policy analysis
print(" Policy Analysis")
print("=" * 30)

# Policy types
print(f"\n POLICY TYPES:")
print(policies['policy_type'].value_counts())

# Policy status
print(f"\n POLICY STATUS:")
print(policies['status'].value_counts())

# Coverage amounts (in thousands)
print(f"\n COVERAGE AMOUNTS (in thousands):")
print(f"Mean: ${policies['coverage_amount'].mean():.2f}K")
print(f"Median: ${policies['coverage_amount'].median():.2f}K")
print(f"Min: ${policies['coverage_amount'].min():.2f}K")
print(f"Max: ${policies['coverage_amount'].max():.2f}K")

# Premium amounts
print(f"\n PREMIUM AMOUNTS:")
print(f"Mean: ${policies['premium_amount'].mean():.2f}")
print(f"Median: ${policies['premium_amount'].median():.2f}")
print(f"Min: ${policies['premium_amount'].min():.2f}")
print(f"Max: ${policies['premium_amount'].max():.2f}")

# Deductibles
print(f"\n DEDUCTIBLES:")
print(f"Mean: ${policies['deductible'].mean():.2f}")
print(f"Median: ${policies['deductible'].median():.2f}")
print(f"Min: ${policies['deductible'].min():.2f}")
print(f"Max: ${policies['deductible'].max():.2f}")


 Policy Analysis

 POLICY TYPES:
policy_type
Auto          458
Home          369
Health        307
Life          207
Disability    153
Name: count, dtype: int64

 POLICY STATUS:
status
Active       1205
Inactive      223
Cancelled      66
Name: count, dtype: int64

 COVERAGE AMOUNTS (in thousands):
Mean: $139.41K
Median: $90.05K
Min: $10.37K
Max: $1277.84K

 PREMIUM AMOUNTS:
Mean: $100.00
Median: $100.00
Min: $100.00
Max: $100.00

 DEDUCTIBLES:
Mean: $2559.20
Median: $2521.66
Min: $106.09
Max: $4997.11


## 4. Document Corpus Analysis

Explore the document corpus containing underwriting guidelines, case studies, and risk assessment methodologies. This corpus will be used for the RAG (Retrieval Augmented Generation) system to provide contextual information for risk scoring.


In [11]:
# Examine document corpus
print(" Document Corpus Overview")
print("=" * 40)
print(f"Total documents: {len(documents)}")

# Document categories
categories = [doc['category'] for doc in documents]
print(f"\n DOCUMENT CATEGORIES:")
print(pd.Series(categories).value_counts())

# Document titles
print(f"\n DOCUMENT TITLES:")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc['title']} ({doc['category']})")

# Sample document content
print(f"\n SAMPLE DOCUMENT CONTENT:")
print("=" * 50)
sample_doc = documents[0]
print(f"Title: {sample_doc['title']}")
print(f"Category: {sample_doc['category']}")
print(f"Content preview: {sample_doc['content'][:200]}...")


 Document Corpus Overview
Total documents: 6

 DOCUMENT CATEGORIES:
underwriting_guidelines    3
case_studies               2
methodology                1
Name: count, dtype: int64

 DOCUMENT TITLES:
1. Auto Insurance Underwriting Guidelines (underwriting_guidelines)
2. Property Insurance Risk Assessment (underwriting_guidelines)
3. Health Insurance Underwriting Criteria (underwriting_guidelines)
4. Case Study: High-Risk Auto Claim (case_studies)
5. Case Study: Property Water Damage (case_studies)
6. Risk Scoring Methodology (methodology)

 SAMPLE DOCUMENT CONTENT:
Title: Auto Insurance Underwriting Guidelines
Category: underwriting_guidelines
Content preview: 
                Auto insurance underwriting guidelines:
                1. Age factor: Drivers under 25 and over 65 have higher risk
                2. Driving record: Clean record for 3+ years prefe...


## 5. Data Quality and Missing Values

Assess data quality by checking for missing values, data types, and overall data integrity. This section ensures the datasets are clean and ready for feature engineering and model development.


In [12]:
# Data quality analysis
print(" Data Quality Analysis")
print("=" * 40)

datasets = {
    'Policyholders': policyholders,
    'Claims': claims,
    'Policies': policies
}

for name, df in datasets.items():
    print(f"\n {name.upper()}:")
    print(f"Shape: {df.shape}")
    print(f"Missing values:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("No missing values!")
    print(f"Data types:")
    print(df.dtypes)


 Data Quality Analysis

 POLICYHOLDERS:
Shape: (1000, 10)
Missing values:
No missing values!
Data types:
policyholder_id       object
age                    int64
gender                object
marital_status        object
annual_income        float64
education_level       object
employment_status     object
credit_score           int64
years_at_address     float64
created_date          object
dtype: object

 CLAIMS:
Shape: (2000, 8)
Missing values:
No missing values!
Data types:
claim_id            object
policyholder_id     object
claim_type          object
claim_amount       float64
severity            object
status              object
claim_date          object
description         object
dtype: object

 POLICIES:
Shape: (1494, 10)
Missing values:
No missing values!
Data types:
policy_id           object
policyholder_id     object
policy_type         object
coverage_amount    float64
premium_amount       int64
status              object
start_date          object
end_date            o

## 6. Key Insights and Summary

Synthesize findings from the data exploration to identify key risk factors, correlations, and patterns. This summary provides actionable insights for building the underwriting risk assessment system.


In [14]:
# Key insights and summary
print(" Key Insights and Summary")
print("=" * 50)

# Risk factors analysis
print("\n RISK FACTORS IDENTIFIED:")
print("1. Credit Score Distribution:")
credit_ranges = pd.cut(policyholders['credit_score'], 
                      bins=[0, 580, 670, 740, 850], 
                      labels=['Poor', 'Fair', 'Good', 'Excellent'])
print(credit_ranges.value_counts())

print("\n2. Income vs Credit Score Correlation:")
correlation = policyholders['annual_income'].corr(policyholders['credit_score'])
print(f"Income-Credit Score Correlation: {correlation:.3f}")

print("\n3. Claims per Policyholder:")
claims_per_policyholder = claims.groupby('policyholder_id').size()
print(f"Average claims per policyholder: {claims_per_policyholder.mean():.2f}")
print(f"Max claims by single policyholder: {claims_per_policyholder.max()}")

print("\n4. High-Risk Indicators:")
high_risk_claims = claims[claims['severity'].isin(['High', 'Critical'])]
print(f"High/Critical severity claims: {len(high_risk_claims)} ({len(high_risk_claims)/len(claims)*100:.1f}%)")

print("\n5. Policy Coverage Analysis:")
avg_coverage = policies['coverage_amount'].mean()
avg_premium = policies['premium_amount'].mean()
print(f"Average coverage: ${avg_coverage:.2f}K")
print(f"Average premium: ${avg_premium:.2f}")
print(f"Premium-to-coverage ratio: {(avg_premium/avg_coverage)*100:.2f}%")

print("\n Data exploration completed successfully!")


 Key Insights and Summary

 RISK FACTORS IDENTIFIED:
1. Credit Score Distribution:
credit_score
Fair         355
Poor         260
Good         212
Excellent    173
Name: count, dtype: int64

2. Income vs Credit Score Correlation:
Income-Credit Score Correlation: -0.015

3. Claims per Policyholder:
Average claims per policyholder: 2.31
Max claims by single policyholder: 8

4. High-Risk Indicators:
High/Critical severity claims: 419 (20.9%)

5. Policy Coverage Analysis:
Average coverage: $139.41K
Average premium: $100.00
Premium-to-coverage ratio: 71.73%

 Data exploration completed successfully!
